IMPORTAZIONE LIBRERIE

In [18]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [19]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M3\FAULT_M3_10


In [20]:
guasto=2

VOLO m3 FAULT 10%

In [21]:
rcou_m3_fault10 = pd.read_csv("RCOU.csv")
rcou_m3_fault10 = rcou_m3_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m3_fault10 = rcou_m3_fault10[((rcou_m3_fault10['C9'] >= 1350) & (rcou_m3_fault10['C10'] >= 1350) & (rcou_m3_fault10['C11'] >= 1350) & (rcou_m3_fault10['C12'] >= 1350) & (rcou_m3_fault10['C13'] >= 1350) & (rcou_m3_fault10['C14']>= 1350))]
rcou_m3_fault10=rcou_m3_fault10.reset_index(drop=True)
display(rcou_m3_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,106891806,1479,1406,1428,1459,1460,1427
1,106907540,1455,1450,1499,1401,1499,1402
2,106920441,1362,1542,1506,1409,1415,1501
3,106931177,1545,1371,1425,1502,1509,1418
4,106942020,1535,1398,1429,1510,1496,1444
...,...,...,...,...,...,...,...
6827,186281436,1556,1438,1522,1478,1504,1495
6828,186291822,1552,1431,1521,1467,1504,1485
6829,186302079,1521,1451,1535,1435,1493,1482
6830,186312025,1502,1450,1546,1397,1494,1458


In [22]:
min=rcou_m3_fault10['TimeUS'][0]
max=rcou_m3_fault10['TimeUS'][len(rcou_m3_fault10)-1]
print(max)
print(min)

186322432
106891806


In [23]:
xkf1_m3_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m3_fault10 = xkf1_m3_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_fault10 = xkf1_m3_fault10[((xkf1_m3_fault10['TimeUS'] >= min) & (xkf1_m3_fault10['TimeUS'] <= max	))]
xkf1_m3_fault10 = xkf1_m3_fault10.reset_index(drop=True)
print(xkf1_m3_fault10)

         TimeUS  Roll  Pitch    Yaw
0     106905542 -0.33   0.35  12.46
1     106918323 -0.39   0.39  12.45
2     106928766 -0.20   0.37  12.46
3     106939536 -0.15   0.37  12.46
4     106950235 -0.17   0.36  12.45
...         ...   ...    ...    ...
7189  186278947 -0.97   1.00   0.99
7190  186289650 -1.03   0.97   1.00
7191  186299874 -1.02   0.92   1.03
7192  186309774 -0.92   0.81   1.03
7193  186319793 -0.76   0.68   1.03

[7194 rows x 4 columns]


In [24]:
att_m3_fault10 = pd.read_csv("ATT.csv")
att_m3_fault10 = att_m3_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_fault10 = att_m3_fault10[((att_m3_fault10['TimeUS'] >= min) & (att_m3_fault10['TimeUS'] <= max	))]
att_m3_fault10=att_m3_fault10.reset_index(drop=True)
print(att_m3_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     106906700     0.04 -0.33     -0.01   0.35   12.39  12.46
1     106919694     0.02 -0.39      0.00   0.39   12.39  12.45
2     106930531     0.00 -0.20      0.00   0.37   12.39  12.46
3     106941454     0.00 -0.15      0.00   0.37   12.39  12.46
4     106951490    -0.02 -0.17      0.00   0.36   12.39  12.45
...         ...      ...   ...       ...    ...     ...    ...
7189  186280650     1.31 -0.97     -0.67   1.00    4.83   0.99
7190  186291048     1.36 -1.03     -0.64   0.97    4.83   1.00
7191  186301226     1.39 -1.02     -0.59   0.92    4.83   1.03
7192  186311233     1.38 -0.92     -0.52   0.81    4.83   1.03
7193  186321744     1.33 -0.76     -0.43   0.68    4.83   1.03

[7194 rows x 7 columns]


In [25]:
from numpy import empty

if len(att_m3_fault10)>len(xkf1_m3_fault10):
    to_add=att_m3_fault10[len(xkf1_m3_fault10):]
    att_m3_fault10=att_m3_fault10[:len(xkf1_m3_fault10)]

if len(xkf1_m3_fault10)>len(att_m3_fault10):
    to_add=xkf1_m3_fault10[len(att_m3_fault10):]

for idx,i in enumerate(att_m3_fault10['Roll']):
    if(xkf1_m3_fault10['Roll'][idx] != i):
        att_m3_fault10['Roll'][idx] = ((att_m3_fault10['Roll'][idx] + xkf1_m3_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m3_fault10['Pitch']):
    if(xkf1_m3_fault10['Pitch'][idx] != i):
        att_m3_fault10['Pitch'][idx] = ((att_m3_fault10['Pitch'][idx] + xkf1_m3_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_fault10['Yaw']):
    if(xkf1_m3_fault10['Yaw'][idx] != i):
        att_m3_fault10['Yaw'][idx] = ((att_m3_fault10['Yaw'][idx] + xkf1_m3_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m3_fault10=pd.concat([att_m3_fault10,to_add])

print(att_m3_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     106906700     0.04 -0.33     -0.01   0.35   12.39  12.46
1     106919694     0.02 -0.39      0.00   0.39   12.39  12.45
2     106930531     0.00 -0.20      0.00   0.37   12.39  12.46
3     106941454     0.00 -0.15      0.00   0.37   12.39  12.46
4     106951490    -0.02 -0.17      0.00   0.36   12.39  12.45
...         ...      ...   ...       ...    ...     ...    ...
7189  186280650     1.31 -0.97     -0.67   1.00    4.83   0.99
7190  186291048     1.36 -1.03     -0.64   0.97    4.83   1.00
7191  186301226     1.39 -1.02     -0.59   0.92    4.83   1.03
7192  186311233     1.38 -0.92     -0.52   0.81    4.83   1.03
7193  186321744     1.33 -0.76     -0.43   0.68    4.83   1.03

[7194 rows x 7 columns]


In [26]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m3_fault10 = esc_0_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_fault10 = esc_0_m3_fault10[((esc_0_m3_fault10['TimeUS'] >= min) & (esc_0_m3_fault10['TimeUS'] <= max))]
esc_0_m3_fault10=esc_0_m3_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m3_fault10 = esc_1_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_fault10 = esc_1_m3_fault10[((esc_1_m3_fault10['TimeUS'] >= min) & (esc_1_m3_fault10['TimeUS'] <= max))]
esc_1_m3_fault10=esc_1_m3_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m3_fault10 = esc_2_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_fault10 = esc_2_m3_fault10[((esc_2_m3_fault10['TimeUS'] >= min) & (esc_2_m3_fault10['TimeUS'] <= max))]
esc_2_m3_fault10=esc_2_m3_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m3_fault10 = esc_3_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_fault10 = esc_3_m3_fault10[((esc_3_m3_fault10['TimeUS'] >= min) & (esc_3_m3_fault10['TimeUS'] <= max))]
esc_3_m3_fault10=esc_3_m3_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m3_fault10 = esc_4_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_fault10 = esc_4_m3_fault10[((esc_4_m3_fault10['TimeUS'] >= min) & (esc_4_m3_fault10['TimeUS'] <= max))]
esc_4_m3_fault10=esc_4_m3_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m3_fault10 = esc_5_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_fault10 = esc_5_m3_fault10[((esc_5_m3_fault10['TimeUS'] >= min) & (esc_5_m3_fault10['TimeUS'] <= max))]
esc_5_m3_fault10=esc_5_m3_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [27]:
#valori di accelerazione e del giroscopio
imu_0_m3_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m3_fault10 = imu_0_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_fault10 = imu_0_m3_fault10[((imu_0_m3_fault10['TimeUS'] >= min) & (imu_0_m3_fault10['TimeUS'] <= max))]

imu_1_m3_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m3_fault10 = imu_1_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_fault10 = imu_1_m3_fault10[((imu_1_m3_fault10['TimeUS'] >= min) & (imu_1_m3_fault10['TimeUS'] <= max))]

imu_2_m3_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m3_fault10 = imu_2_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_fault10 = imu_2_m3_fault10[((imu_2_m3_fault10['TimeUS'] >= min) & (imu_2_m3_fault10['TimeUS'] <= max))]

imu_m3_fault10 = pd.concat((imu_0_m3_fault10, imu_1_m3_fault10, imu_2_m3_fault10))
imu_m3_fault10=imu_m3_fault10.groupby(imu_m3_fault10.TimeUS, as_index=False).mean()

display(imu_m3_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,106893398,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562
1,106896186,-0.205686,-0.056071,0.012061,0.153629,0.735929,-9.858738
2,106898723,-0.079848,0.040186,0.001080,0.093860,0.851389,-9.918224
3,106902129,0.180783,0.050828,-0.016770,0.214891,1.032160,-9.818566
4,106904697,0.305814,-0.087739,-0.038796,0.212236,0.727475,-9.630442
...,...,...,...,...,...,...,...
27341,186311070,0.089517,-0.180582,0.027792,0.618502,1.844759,-12.362567
27342,186313759,0.151039,-0.174859,-0.009015,0.656361,2.363123,-12.738627
27343,186316447,0.319917,-0.200447,-0.010393,0.736293,2.474347,-12.872283
27344,186318718,0.269170,-0.224830,-0.001907,0.871600,2.249914,-12.945027


In [28]:
#sincronizzazione dei tempi tra att e imu_2
m3_fault10 = pd.merge_ordered(imu_m3_fault10,att_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_0_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_1_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_2_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_3_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_4_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_5_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")
m3_fault10 = pd.merge_ordered(m3_fault10, rcou_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")

In [29]:
index_to_remove=[]
for ind in att_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m3_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

7194
7864
8532
9207
9873
10552
11208


In [30]:
m3_fault10 = m3_fault10[~m3_fault10.TimeUS.isin(index_to_remove)]
m3_fault10["TimeUS"] = m3_fault10["TimeUS"] - m3_fault10.iloc[0]["TimeUS"]
m3_fault10["TimeUS"] = m3_fault10["TimeUS"].astype(int)
m3_fault10["Guasto"] = guasto

In [31]:
from datetime import timedelta
m3_fault10=m3_fault10.reset_index(drop=True)
m3_fault10['TimeUS'] = pd.to_datetime(m3_fault10['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2
1,00:00:00.002788,-0.205686,-0.056071,0.012061,0.153629,0.735929,-9.858738,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2
2,00:00:00.005325,-0.079848,0.040186,0.001080,0.093860,0.851389,-9.918224,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2
3,00:00:00.008731,0.180783,0.050828,-0.016770,0.214891,1.032160,-9.818566,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2
4,00:00:00.011299,0.305814,-0.087739,-0.038796,0.212236,0.727475,-9.630442,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27341,00:01:19.417672,0.089517,-0.180582,0.027792,0.618502,1.844759,-12.362567,1.39,-1.02,-0.59,...,1.99,5457.0,3.08,1521.0,1451.0,1535.0,1435.0,1493.0,1482.0,2
27342,00:01:19.420361,0.151039,-0.174859,-0.009015,0.656361,2.363123,-12.738627,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2
27343,00:01:19.423049,0.319917,-0.200447,-0.010393,0.736293,2.474347,-12.872283,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2
27344,00:01:19.425320,0.269170,-0.224830,-0.001907,0.871600,2.249914,-12.945027,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2


## Analisi nel tempo e in frequenza

In [32]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [33]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})','guasto'])
    for i in range(int(len(m3_fault10)/350)):
        V1=m3_fault10[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2,'guasto':guasto}
        tab=tab.append(val,ignore_index=True)
    return tab

In [34]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_19912\1844060170.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_19912\1844060170.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),guasto_x,...,guasto_y,<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),guasto
0,0.0,0.000856,0.016478,0.559424,0.128187,93.0,63.0,5.599285,2.444539,2.0,...,2.0,3.528917,0.043591,-0.986998,3.535071,131.0,130.0,28.264201,5.843806,2.0
1,1.0,0.000295,0.016542,0.533523,0.128433,91.0,71.0,5.538370,5.451980,2.0,...,2.0,3.528949,0.043468,-0.981609,3.535085,131.0,130.0,28.086826,6.008924,2.0
2,2.0,-0.000512,0.016725,0.507932,0.129142,79.0,103.0,7.038925,5.378759,2.0,...,2.0,3.528980,0.043344,-0.976221,3.535099,129.0,128.0,27.898393,6.164138,2.0
3,3.0,-0.001575,0.017078,0.542756,0.130506,105.0,82.0,5.116888,5.009156,2.0,...,2.0,3.529011,0.043222,-0.970832,3.535113,129.0,128.0,27.699075,6.309410,2.0
4,4.0,-0.002509,0.017339,0.532814,0.131514,109.0,85.0,4.890383,3.057094,2.0,...,2.0,3.529042,0.043100,-0.965443,3.535126,133.0,132.0,27.489051,6.444725,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,73.0,-0.036091,0.022036,-0.035691,0.152598,122.0,98.0,6.523278,5.697257,2.0,...,2.0,3.621014,0.093401,0.479420,3.633858,143.0,141.0,24.147716,12.266073,2.0
74,74.0,-0.036563,0.022078,-0.049558,0.152850,118.0,119.0,6.471940,5.393019,2.0,...,2.0,3.623035,0.094916,0.477412,3.636080,133.0,131.0,24.394029,12.144172,2.0
75,75.0,-0.036709,0.022035,-0.045956,0.152745,128.0,129.0,6.546225,5.294452,2.0,...,2.0,3.625047,0.096417,0.472427,3.638290,139.0,137.0,24.619873,11.925637,2.0
76,76.0,-0.037008,0.022022,-0.050041,0.152774,102.0,130.0,7.505378,6.542461,2.0,...,2.0,3.627049,0.097903,0.464828,3.640489,145.0,143.0,24.825353,11.616564,2.0


In [35]:
path_file = path_file.replace(r"M3\FAULT_M3_10", "")
os.chdir(path_file)
m3_fault10.to_csv('m3_fault10.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
